# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Цель исследования - проверить четыре гипотезы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели влияют на его возврат в срок?

Ход исследования

Данные о поведении заемщиков представлены в файле https://code.s3.yandex.net/datasets/data.csv. О качестве данных ничего неизвестно, поэтому перед проверкой гипотез понадобится обзор данных.
Данные будут проверены на ошибки и будет оценено их влияние на исследование. Затем, на этапе предобработки будет проведен поиск возможности исправления самых критичных ошибок данных.

Таким образом, иследование пройдет в три этапа:

1. Обзор данных.
2. Предобработка данных.
3. Проверка гипотез.
4. Общий вывод.


## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd #загрузка библиотеки
df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')#загрузка файла
df.head(10)# выводим первые строки таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()# выводим сводную информации о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.describe()# выводим сводную информации о таблице

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Итак, в таблице двенадцать стобцов. Типы данных в столбцах: object, int64, float64.

Согласно документации к данным:

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

В названиях колонок нарушения стиля не видны.

Количество значений в столбцах различатся. Значит, в данных есть пропущенные значения.

**Вывод**

В каждой строке таблицы - данные статистики о платежеспособности клиентов банка. Часть колонок описывает клиентов с точки зрения возраста, семейного положения и количества детей. Остальные данные рассказывают о доходе клиентов с точки зрения статуса в компании, стаже работы и образовании.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но, в данных встречаются следующие проблемы:
1. Пропуски в столбцах total_income days_employed.
2. Дубликаты, которые отличаются регистром нескольких букв.
3. В столбце total_income cодержатся сведения с большим количеством знаков после запятой, что осложняет проверку гипотез для всего датасета.
4. В столбце total_income содержится огромное количество уникальных значений, для чего необходимо будт провести категоризацию данных.
5. В столбце purpose встречаются неявные дубликаты. 

После предобработки данных исследование станет более точным.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
df.isna().sum()# определим пропущенные значения в таблице

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Полученные при обработке данных пропуски составляют одинаковое количество пропусков в размере 2 174 пропуска в двух столбцах, а именно в столбце days_employed и в столбце total_income. Указанную закономерность можно интерпретировать как то, что указанные данные отсутствют не по причине ошибочных действий и не относятся к категории 'полностью случайные'. 
На первый взгляд, полученные пропуски в столбце days_employed не влияют на исследование, которое заключается в том, чтобы оценить влияет ли семейное положение и количество детей заемщика на факт погашения кредита в срок. 

В столбце total_income отражена сумма дохода клиента, которую в дальнейшем будем использовать при проверке гипотезы о зависимости между уровнем дохода и возвратом кредита в срок. 
Проведем анализ столбца days_employed, с целью выявить закономерности в ошибочных данных и исправить их. Для этого найдем max(), min() этого солбца и посмотрим на другие показатели заемщиков с этими значениями.

In [5]:
df_max = df['days_employed'].max()# определяем максимум
df_max

401755.40047533

Анализ максимального значения столбца показал, что 401755.40047533 = 1100,69973 лет. 
При этом, показатели в днях содержат значения после запятой.

In [6]:
df[df['days_employed'] == df_max]# используем логическую индексацию для  получения элементов по определенному условию 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью


Проведем анализ заемщика 6954  и установим, что для мужчины возраста 56 лет максимальный стаж работы может составлять 38 лет.
8760 = 365*24
45 = 401755.40047533/8760
Проведя расчеты, можно установить, что стаж работы указанного заемщика составляет 45 лет(то есть заемщик начал работать официально с 11 лет). Указанное не может соответствовать истине.
Таким образом, определить ошибку в данномм показателе не представляется возможным.

Рассмотрим ситуацию с отрицательными значениями. Сделаем предположение, что отрицательное значение случайное и ошибочное.

In [7]:
df_min = df['days_employed'].min()#определяем минимум
df_min

-18388.949900568383

In [8]:
df[df['days_employed'] == df_min]# используем логическую индексацию для  получения элементов по определенному условию 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью


In [9]:
print(18388.949900568383/8760)# посчитаем возможный стаж работы в годах, сделав предоложение, что отрицательное значение ошибочно

2.0991951941288107


Проведем анализ заемщика 16335  и установим, что для мужчины возраста 61 лет максимальный стаж работы может составлять 43 года.
Стаж работы указанного клиента составит:
50.38068465909146 = 18388.949900568383/365

Если сделать предположение, что данные занесены в часах, то стаж работы указанного клиента составит:
8760 = 365*24
2.0991951941288107 = 18388.949900568383/8760

Указанное не может соответствовать истине.

Таким образом, определить ошибку в данном показателе также не представляется возможным.
В данной связи принято решение не использовать в расчетах данные из столбца days_employed.

С целью проверки наличия некорректных значений в столбце children, посчитаем для него уникальные значения.


In [10]:
df['children'].value_counts()#посчитаем количество уникальных значений

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце children найдены значения, отличающиеся от обычных: -1 ребенок, 20 детей.
Поскольку, согласно представленным данным 76  семей имеют 20 детей, при этом количество семей, имеющих 20 детей, по данным из сети Интернет в России очень редкое явление, то примем их за случайные ошибки в данных.

Методом replace() произведем замену показателей на 1 и 2 соответственно. 

In [11]:
df['children'] = df['children'].replace(-1, 1)# заменим значения в столбце
df['children'] = df['children'].replace(20, 2)
df['children'].value_counts()#посчитаем уникальные значения после замены данных

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Поскольку при проводимом исследовании будут использованы данные из столбца total_income, проведем группировку медианных значений столбца total_income по типу занимаемой должности.

In [12]:
df.groupby('income_type')['total_income'].median()#посчитем медианные значения

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Заполним пропущенные значения в столбце total_income медианным значением.

In [13]:
df_transformed = df.groupby('income_type')['total_income'].transform('median')# заполним медианным значением пропцщенные значения в столбце по заданным условиям и выведем пропущенные занчения
df['total_income'] = df['total_income'].fillna(df_transformed)
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Выведем уникальные значения столбца dob_years.

In [14]:
df['dob_years'].value_counts()#посчистаем уникальные значения

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце 'dob_years' выявлены нулевые значения. Выведем 5 строк со значением 0 и проведем анализ указанных строк.

In [15]:
df[df['dob_years'] == 0].head()#используя логическую индексацию выведем 5 строк, соответствующих условию 0 в столбце dob_years

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


Никакого логического объяснения указанным ошибочным данным нет. 

Проведем замену данных значения 0 на медианные значения в столбце dob_years.
В данной связи используя метод replace() проведем замену значения 0 на значение None. И используя метод transform() проведем замену None на медианные значения. 

In [16]:
df['dob_years'] = df['dob_years'].replace(0, None)# проведем замену значения 0 на медианные значения в столбце dob_years и выведем уникальные значения после преобразования
df_transformed = df.groupby('income_type')['dob_years'].transform('median')
df['dob_years'] = df['dob_years'].fillna(df_transformed)
df['dob_years'].value_counts()

35    618
41    611
40    610
34    604
38    603
42    602
33    584
39    574
31    562
36    559
44    549
29    547
30    544
37    542
48    540
50    519
43    515
32    513
49    511
28    505
45    497
27    494
56    490
52    485
47    482
54    482
46    478
53    463
57    463
58    462
51    451
59    448
55    445
26    411
60    379
25    357
61    355
62    352
63    269
64    265
24    264
23    255
65    196
66    183
22    183
67    170
21    112
68    100
69     85
70     65
71     58
20     51
72     33
19     14
73      9
74      6
75      1
Name: dob_years, dtype: int64

С целью проверки наличия некорректных значений в столбце gender, посчитаем для него уникальные значения.

In [17]:
df['gender'].value_counts()#выведем уникальные значения

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце 'gender' выявлено 1 значение пола XNA. Выведем 5 строк со значением 0 и проведем анализ указанных строк.

In [18]:
df[df['gender'] == 'XNA']#используя логическую индексацию посмотрим на строку, удовлетворяющую определенным условиям

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Никакого логического объяснения указанным ошибочным данным нет. В данной связи проведем замену значения XNA на значение М.

In [19]:
df['gender'] = df['gender'].replace('XNA', 'M')# проведем замену значения XNA на значение М
df['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

**Вывод**

В ходе обработки пропусков:
1. в столбце total_income заменили пропуски на медианные значения.
2. в столбце dob_years заменили возрат = 0 на медианные значения возраста
3. в столбце gender заменили неизместное значение XNA на значение M
4. в столбце children заменили значения -1 и 20.

Также провели анализ возможности замены значений столбца days_employed, обратили внимание на неадекватность стажа и отрицательные значения. При использовании указанных значения данные были бы сильно искажены, в связи с чем было принято решение не использовать указанные данные в расчетах.


### Замена типа данных

In [20]:
df.info()# сводная информация о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Установлено, что столбцы days_employed и total_income распознаны как строки - тип object. Это значит, что в столбцах есть нечисловые значения, или пропуски, а они помешают нашим расчетам. Учитывая то, что столбец total_income необходим для проверки одной из гипотез, с целью укрупнения данных, вещественный тип данных был заменен на целочисленный с применением метода astype().

In [21]:
df['total_income'] = df['total_income'].astype('int')# Перевод  вещественный тип данных в целочисленный тип данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**


Замена типа данных обнаружила следующую проблему:

1.Столбцы days_employed и total_income распознаны как строки - тип object. 

Чтобы упростить работу с таблицей вещественный тип данных был заменен на целочисленный с применением метода astype().

### Обработка дубликатов

Возможными причинами появления дубликатов в данных можно считать повторные представления данных, либо неправильное соединение данных из разных источников, а также ошибки пользователя при занесении информации.

Используя метод duplicated(), в сочетании с методом sum(), установим, существуют ли в таблице одинаковые строки с идентичной информацией. 

In [22]:
df.duplicated().sum()# выведем количество дубликатов

55

Большое количество повторов раздувает размер таблицы, а значит, увеличивает время обработки данных. В данной связи, используя метод df.drop_duplicates().reset_index(drop = True), удалим дубликаты.

In [23]:
df = df.drop_duplicates().reset_index(drop=True)#удалим дубликаты
df.duplicated().sum()

0

Кроме того, в столбце education установлены одинаковые по смыслу категории, которые отличаются регистром нескольких букв.

In [24]:
df['education'].value_counts()#выведем уникальные значения

среднее                13704
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Чтобы учесть такие дубликаты, все символы в строке приведем к нижнему регистру вызовом в Pandas метода str.lower():

In [25]:
df['education'] = df['education'].str.lower()# Приведем строки к нижнему регистру
df['education'].value_counts()

среднее                15187
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод**

Обработка дубликатов показала следующие две проблемы:

1. Полные дубликаты
2. Вид дубликатов отличается регистром нескольких букв.

Возможными причинами появления дубликатов в данных можно считать повторные представления данных, либо неправильное соединение данных из разных источников, а также ошибки пользователя при занесении информации.

В ходе обработки дубликатов, используя метод df.drop_duplicates().reset_index(drop = True), удалили полные дубликаты, а также все символы в строке привели к нижнему регистру вызовом в Pandas метода str.lower().

Без дубликатов исследование станет более точным.

### Лемматизация

Выведем на экран список уникальных целей кредита в алфавитном порядке. 

In [26]:
df['purpose'].sort_values().unique()# выведем все уникальные значения в столбце

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Выведем список лемматизированных слов(сведенных к лемме), с использованием метода join().

In [27]:
from pymystem3 import Mystem # Получение стеммера/лемматизатора для слов на русском 
from collections import Counter # библиотека для подсчета различных значений в списке
purpose = '_'.join (df['purpose'].sort_values().unique())# объединим список строк с помощью определенного указателя отсортированных уникальных значений по возрастанию
display(purpose)

'автомобили_автомобиль_высшее образование_дополнительное образование_жилье_заняться высшим образованием_заняться образованием_на покупку автомобиля_на покупку подержанного автомобиля_на покупку своего автомобиля_на проведение свадьбы_недвижимость_образование_операции с жильем_операции с коммерческой недвижимостью_операции с недвижимостью_операции со своей недвижимостью_покупка жилой недвижимости_покупка жилья_покупка жилья для сдачи_покупка жилья для семьи_покупка коммерческой недвижимости_покупка недвижимости_покупка своего жилья_получение высшего образования_получение дополнительного образования_получение образования_приобретение автомобиля_профильное образование_ремонт жилью_свадьба_свой автомобиль_сделка с автомобилем_сделка с подержанным автомобилем_строительство жилой недвижимости_строительство недвижимости_строительство собственной недвижимости_сыграть свадьбу'

Посчитаем число лемматизированных слов в тексте.

In [28]:

m = Mystem()
purpose_lemmas = m.lemmatize(purpose)# Лемматизация строки с русским текстом
Counter(purpose_lemmas)# Поcчитаем различные значения в списке

Counter({'автомобиль': 9,
         '_': 37,
         'высокий': 3,
         ' ': 59,
         'образование': 9,
         'дополнительный': 2,
         'жилье': 7,
         'заниматься': 2,
         'на': 4,
         'покупка': 10,
         'подержать': 2,
         'свой': 4,
         'проведение': 1,
         'свадьба': 3,
         'недвижимость': 10,
         'операция': 4,
         'с': 5,
         'коммерческий': 2,
         'со': 1,
         'жилой': 2,
         'для': 2,
         'сдача': 1,
         'семья': 1,
         'получение': 3,
         'приобретение': 1,
         'профильный': 1,
         'ремонт': 1,
         'сделка': 2,
         'строительство': 3,
         'собственный': 1,
         'сыграть': 1,
         '\n': 1})

Основными категориями являются: автомобиль, образование, свадьба, а также жилье (недвижимость). С использованием стемминга разделим указанные лемматизированные слова по категориям.

In [29]:
from nltk.stem import SnowballStemmer# Получение стеммера для русского языка
list_wed = []
russian_stemmer = SnowballStemmer('russian')
for query_wed in df['purpose'].sort_values().unique():
    for word in query_wed.split():
        stemmed_word = russian_stemmer.stem(word)# Получение стема от слова на русском
        if stemmed_word =='свадьб':
            list_wed.append(query_wed)
list_wed

['на проведение свадьбы', 'свадьба', 'сыграть свадьбу']

In [30]:
list_auto = []
for query_auto in df['purpose'].sort_values().unique():
    for word in query_auto.split():
        stemmed_word = russian_stemmer.stem(word)
        if stemmed_word =='автомоб' or stemmed_word =='автомобил':
            list_auto.append(query_auto)
list_auto

['автомобили',
 'автомобиль',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'приобретение автомобиля',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем']

In [31]:
list_edu = []
for query_edu in df['purpose'].sort_values().unique():
    for word in query_edu.split():
        stemmed_word = russian_stemmer.stem(word)
        if stemmed_word =='образован':
            list_edu.append(query_edu)
list_edu

['высшее образование',
 'дополнительное образование',
 'заняться высшим образованием',
 'заняться образованием',
 'образование',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'профильное образование']

In [32]:
list_prop = []
for query_prop in df['purpose'].sort_values().unique():
    for word in query_prop.split():
        stemmed_word = russian_stemmer.stem(word)
        if stemmed_word =='недвижим' or stemmed_word == 'жил':
            list_prop.append(query_prop)
        
list_prop

['жилье',
 'недвижимость',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'ремонт жилью',
 'строительство жилой недвижимости',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недвижимости']

Методом replace() устраняем  неявные дубликаты.

In [33]:
df['purpose'] = df['purpose'].replace(list_prop, 'операции с недвижимостью')
df['purpose'] = df['purpose'].replace(list_wed, 'на проведение свадьбы')
df['purpose'] = df['purpose'].replace(list_edu, 'получение образования')
df['purpose'] = df['purpose'].replace(list_auto, 'покупка автомобиля')
df.groupby('purpose').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income
purpose,,,,,,,,,,,
на проведение свадьбы,2335,2099,2335,2335,2335,2335,2335,2335,2335,2335,2335
операции с недвижимостью,10813,9758,10813,10813,10813,10813,10813,10813,10813,10813,10813
покупка автомобиля,4308,3897,4308,4308,4308,4308,4308,4308,4308,4308,4308
получение образования,4014,3597,4014,4014,4014,4014,4014,4014,4014,4014,4014


**Вывод**

Лемматизация данных показала, что основными категориями являются: автомобиль, образование, свадьба, а также жилье (недвижимость). С использованием стемминга разделили указанные лемматизированные слова по категориям. Методом replace() устраняем неявные дубликаты. В итоге получили 4 категории целей кредита, которые в дальнейшем удобно использовать для проверки гипотезы о влиянии разных целей на возврат кредита в срок.

### Категоризация данных

Поскольку в столбце dob_years содержится очень большое количество разрозненных данных, необходимо провести категоризацию
данных для целей провекри гипотез. 

In [34]:
df['dob_years'].sort_values().unique()# выведем все уникальные значения в столбце

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75])

In [35]:
df.groupby('income_type').agg({'dob_years':  ['min', 'max', 'median']})# Применим несколько операций к столбцу при группировке

dob_years           
                      min max median
income_type                         
безработный            31  45   38.0
в декрете              39  39   39.0
госслужащий            19  75   40.0
компаньон              19  74   39.0
пенсионер              21  74   60.0
предприниматель        27  58   42.5
сотрудник              19  74   39.0
студент                22  22   22.0

In [36]:
display(df['dob_years'].quantile(0.25))
display(df['dob_years'].quantile(0.5))
display(df['dob_years'].quantile(0.75))

33.0

43.0

53.0

In [37]:
def age_group(age):#с помощью функции проведем разделение на группы
    if age < 33:
        return 'до 33'
    if age <= 42:
        return 'до 42'
    if age <= 53:
        return 'до 53'
    return 'остальные'
df['dob_years_kvant'] = df['dob_years'].apply(age_group)#добавим новый столбец к таблице

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_kvant
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,операции с недвижимостью,до 42
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,покупка автомобиля,до 42
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,операции с недвижимостью,до 42
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,получение образования,до 33
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,на проведение свадьбы,до 53


В данной связи проведем разделение возраста заемщиков, основываясь на данных из столбца dob_years 
(а именно: 'min', 'max','median').

до 22  (возраст студента)

до 43 (медианное значение)

до 60 (медианное значение пенсионера)

остальные.

In [38]:

def age_group(age):#с помощью функции проведем разделение на группы
    if age < 22:
        return 'до 22'
    if age <= 30:
        return 'до 30'
    if age <= 55:
        return 'до 55'
    return 'остальные'
df['dob_years_group'] = df['dob_years'].apply(age_group)#добавим новый столбец к таблице

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_kvant,dob_years_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,операции с недвижимостью,до 42,до 55
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,покупка автомобиля,до 42,до 55
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,операции с недвижимостью,до 42,до 55
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,получение образования,до 33,до 55
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,на проведение свадьбы,до 53,до 55


In [39]:
df.groupby('dob_years_group')['total_income'].sum()#выведем сумму значений для столбца dob_years_group

dob_years_group
до 22          24377944
до 30         574399685
до 55        2306315211
остальные     643821654
Name: total_income, dtype: int64

In [40]:
df.groupby('dob_years_kvant')['total_income'].sum()#выведем сумму значений для столбца dob_years_kvant

dob_years_kvant
до 33         779905178
до 42        1040819775
до 53         941770569
остальные     786418972
Name: total_income, dtype: int64

Поскольку в столбце total_income содержится очень большое количество разрозненных данных, необходимо провести категоризацию данных для целей провекри гипотезы о нличии/отсутствии зависимости между уровнем дохода и возвратом кредита в срок. 

In [41]:
df['total_income'].sort_values().unique()#выведем отсортированные уникальные значения

array([  20667,   21205,   21367, ..., 1726276, 2200852, 2265604])

In [42]:
df.groupby('income_type').agg({'total_income': ['min', 'max', 'median']})

total_income                 
                         min      max  median
income_type                                  
безработный            59956   202722  131339
в декрете              53829    53829   53829
госслужащий            29200   910451  150447
компаньон              28702  2265604  172357
пенсионер              20667   735103  118514
предприниматель       499163   499163  499163
сотрудник              21367  1726276  142594
студент                98201    98201   98201

In [43]:
display(df['total_income'].quantile(0.25).astype('int'))
display(df['total_income'].quantile(0.5).astype('int'))
display(df['total_income'].quantile(0.75).astype('int'))

107652

142594

195775

В данной связи проведем разделение дохода на 3 категории, основываясь на данных из столбца purpose (а именно: 'min', 'max','median'), поскольку сумма кредита зависит от его цели и от уровня дохода:
- до 22 тыс. (минимальная сумма дохода в зависимости от цели кредита)
- до 150 тыс. (медианное значение в зависимости от цели кредита)
- свыше 1 млн (сумма, свыше которой доход будет отнесен к категории максимальный).

In [44]:
def total_income_group(total_income):
    if total_income < 22000:
        return 'до 22 тыс.'
    if total_income <= 150000:
        return 'до 150 тыс.'
    return 'свыше 1 млн'
df['total_income_group'] = df['total_income'].apply(total_income_group)

df.groupby('total_income_group')[['total_income']].count()


,total_income
total_income_group,
до 150 тыс.,11632
до 22 тыс.,5
свыше 1 млн,9833


In [45]:
def total_income_kvant(total_income):
    if total_income < 107652:
        return 'до 108 тыс.'
    if total_income <= 142594:     
        return 'до 143 тыс.'
    if total_income <= 195775:
        return 'до 196 тыс.'
    return 'свыше 196'
df['total_income_kvant'] = df['total_income'].apply(total_income_kvant)
df.groupby('total_income_kvant')[['total_income']].count()

,total_income
total_income_kvant,
до 108 тыс.,5368
до 143 тыс.,5489
до 196 тыс.,5245
свыше 196,5368


**Вывод**


В ходе предобработки данных проведена категоризация данных:

1. в целях проверки гипотез проведено разделение возраста заемщиков, основываясь на данных из столбца dob_years 
проведена категоризация данных столбца dob_years на 4 категории, с целью укрупнения данных:

- до 22  (возраст студента)
- до 43 (медианное значение)
- до 60 (медианное значение пенсионера)
- остальные.

2. в целях провекри гипотезы о нличии/отсутствии зависимости между уровнем дохода и возвратом кредита в срок проведена категоризация данных столбца total_income на 3 категории, с целью укрупнения данных:
- до 22 тыс. 
- до 150 тыс.
- свыше 1 млн


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

При проверке первой гипотезы проверим, есть ли зависимость между наличием детей и возвратом кредита в срок. 
Для этого сначала выделим каждый из расчетов по отдельности, а потом объеденим указанные расчеты:
1. Сколько всего заемщиков с определенным количеством детей.
2. Сколько всего заемщиков вышли на просрочку.


Оценим количество заемщиков для семей без детей, а также для семей с разным количеством детей.

In [46]:
max_percent = df.groupby('children')[['debt']].count()#посчитаем и выведем количество значений для столбца children
display(max_percent)
display(max_percent['debt'][0] / max_percent['debt'].sum() * 100)#выведем процент по формуле для значения в нулевой ячейке столбца debt в таблице max_percent

,debt
children,
0,14106
1,4856
2,2128
3,330
4,41
5,9


65.70097810898929

Из представленных данных следует, что большинство заемщиков не имеют детей (65,7%). А также указанная группировка показывает, что количество заемщиков с большим количеством детей меньше, чем количество заемщиков без детей.

Теперь сгруппируем данные по заемщикам, которые вышли на просрочку.

In [47]:
max_percent= df.groupby('debt')[['children']].count()#посчитаем и выведем количество значений для столбца debt
display(max_percent)
display(max_percent['children'][1] / max_percent['children'].sum() * 100)#выведем процент по формуле для значения в первой ячейке столбца children в таблице max_percent

,children
debt,
0,19729
1,1741


8.108989287377737

Указанные сгруппированные данные показывают, что из всего количества заемщиков, процент вышедших на просрочку составляет 8.1%.

Теперь сформируем сводную таблицу, которая посчитает зависимость между наличием детей и возвратом кредита в срок.

In [48]:
def per_cent_n(debtors, lenders):#с помощью функции посчитаем процент заемщиков с просроченной задолженностью от общего количества заемщиков
    per_cent = debtors/(debtors + lenders)*100
    return per_cent

def make_dependence(dependence):#c помощью функции посчитаем процент заемщиков с просроченной заделженностью по сводной таблице
    df_pivot_debt = df.pivot_table(index=df[dependence], columns= 'debt', values='education_id', aggfunc='count')
    df_pivot_debt[1] = df_pivot_debt[1].fillna(0)
    df_pivot_debt['percent'] = [per_cent_n(df_pivot_debt[1][i], df_pivot_debt[0][i]) for i in range(len(df_pivot_debt))]
    df_pivot_debt = df_pivot_debt[['percent']]
    display(df_pivot_debt)

make_dependence('children')


debt,percent
children,
0,7.535800
1,9.163921
2,9.492481
3,8.181818
4,9.756098
5,0.000000


Из полученных данных видно, что максимальный процент просрочки у заемщиков, имеющих 4 детей, а минимальный процент просрочки у заемщиков с 5 детьми.
При этом заемщики с 3 детьми чаще возвращают кредит в срок, чем заемщики с 1 ребенком.

Попробуем установить влияет ли семейное положение должников, а также их образование, пол, или возраст на зависимость между количеством детей.

При помощи функции создадим сводные таблицы pivot_table() с двумя уровнями.

Рассмотрим как влияет наличие детей на возврат кредита в срок у должников с разным уровнем образования.

In [49]:

def make_double_dependence(dep1, dep2):# c помощью функции создадим сводные таблицы pivot_table() с двумя уровнями.
    df_pivot = df.pivot_table(index=df[[dep1, dep2]], columns = 'debt', values='education_id', aggfunc='count')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    df_pivot['percent'] = [per_cent_n(df_pivot[1][i], df_pivot[0][i]) for i in range(len(df_pivot))]
    df_pivot = df_pivot[[dep1, dep2, 'percent']]
    df_filtered = df_pivot.pivot_table(index = df[[dep1]], columns = dep2)
    return df_filtered
    
make_double_dependence('children', 'education')



debt        percent                                                         
education    высшее  начальное неоконченное высшее    среднее ученая степень
children                                                                    
0          5.306748   7.936508            7.613169   8.242353            0.0
1          5.413534  16.666667           11.363636  10.428702            0.0
2          5.096661  24.000000           12.162162  10.821918            NaN
3          4.819277   0.000000           25.000000   9.090909            0.0
4          0.000000        NaN                 NaN  12.500000            NaN
5               NaN   0.000000                 NaN   0.000000            NaN

Как видно из таблицы выше, просрочка отсутствет у должников с ученой степенью. При этом, у должников с высшим образованием процент
просроченной задолженности также ниже (от 0% до 5.4%) процента просроченнной задолженности всех заемщиков (8.1%).
Кроме того, у должников с неоконченным высшим образованием  процент просроченной задолженности возрастает с увеличением количества детей.

Рассмотрим как влияет наличие детей на возврат кредита в срок у должников разного пола.

In [50]:
make_double_dependence('children', 'gender')


debt        percent           
gender            F          M
children                      
0          6.210007  10.299585
1          7.882089  11.469741
2         10.590944   7.757576
3          8.673469   7.462687
4          3.571429  23.076923
5          0.000000   0.000000

Как видно из данных, представленных в таблице выше, самое большое количество просроченной задолженности у заемщиков мужского пола, имеющих 4 детей (23.076923%), при этом просроченной задолженности ни у заемщиков мужского пола, ни у заемщиков женского пола, имеющих 5 детей, нет.

Рассмотрим как влияет наличие детей на возврат кредита в срок у должников с разным семейным статусом.

In [51]:
make_double_dependence('children', 'family_status')#выведем функцию со сводными таблицами с двумя уровнями.


debt                        percent                                            \
family_status Не женат / не замужем в разводе вдовец / вдова гражданский брак   
children                                                                        
0                          9.283820  7.015306       6.257379         8.354615   
1                         11.453744  6.645570       8.641975        11.788212   
2                         11.904762  9.638554      12.500000         9.295775   
3                         12.500000  9.090909       0.000000        14.285714   
4                         50.000000  0.000000       0.000000         0.000000   
5                               NaN       NaN            NaN         0.000000   

debt                           
family_status женат / замужем  
children                       
0                    6.905782  
1                    8.222370  
2                    9.355247  
3                    6.827309  
4                   10.344828  
5                    0.000000

Как видно из данных, приведенных в таблице выше, самое большое количество просроченной задолженности у заемщиков со статусом 
'Не женат / не замужем', и имеющих 4 детей (50%), при этом просроченной задолженности ни у заемщиков мужского пола, ни у заемщиков женского пола, имеющих 5 детей, нет.

Рассмотрим влияние наличия детей на возврат кредита в срок у должников разного возраста.

In [52]:
make_double_dependence('children', 'dob_years_group')

debt               percent                               
dob_years_group      до 22      до 30     до 55 остальные
children                                                 
0                11.564626  10.845771  7.658340  5.493413
1                 7.407407  10.886320  8.965912  5.666667
2                 0.000000  11.283186  9.046455  8.108108
3                      NaN   6.000000  8.695652  0.000000
4                      NaN  16.666667  8.571429       NaN
5                      NaN        NaN  0.000000  0.000000

Как видно из данных, приведенных в таблице выше, начиная с 30 лет у заемщиков, по сформированным возрастным группам,  снижается процент просроченной задолженности, вне зависимости от количества детей.

**Вывод**

В ходе анализа представленных в таблице данных, о наличии зависимости между наличием детей и возвратом кредита в срок, установлено следующее. Всего заемщиков - 21 470 человек. Из них заемщиков, вышедших на просрочку - 1 741 человек. Процент просроченной задолженности по всей выборке составляет 8,1%.

По результатам проведенного анализа установлено, что наибольший процент просроченной задолженности у заемщиков с 4 детьми и составляет 9,75 %, при этом задолженность отсвутствует у заемщиков с 5-ю детьми.

Кроме того, в ходе проведенного анализа установлено, что:

1. просрочка отсутствет у должников с ученой степенью (вне зависимости от количества детей), а также процент просроченной задолженности у должников с высшим образованием ниже, чем процент просроченной задолженности всех заемщиков;

2. у должников с неоконченным высшим образованием процент просроченной задолженности возрастает с увеличением количества детей;

3. начиная с 30 лет у заемщиков, по сформированным возрастным группам, снижается процент просроченной задолженности, вне зависимости от количества детей;

4. самое большое количество просроченной задолженности у заемщиков мужского пола, имеющих 4 детей (23.076923%);

5. самое большое количество просроченной задолженности у заемщиков со статусом 'Не женат / не замужем', и имеющих 4 детей (50%).

Таким образом, гипотеза о зависимости между наличием детей и возвратом кредита в срок не подтверждается.
Если зависимость и существует, то на основной массе должников указанного датасета она не заметна.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

При рассмотрении второй гипотезы проверим, есть ли зависимость между семейным положением и возвратом кредита в срок. 
Для этого сначала рассчитаем сколько всего заемщиков с определенным видом семейного положения.

Оценим количество заемщиков для заемщиков с разным семейным положением.

In [53]:
max_percent = df.groupby('family_status')[['debt']].count()
display(max_percent)
display(max_percent['debt'][4] / max_percent['debt'].sum() * 100)

,debt
family_status,
Не женат / не замужем,2810
в разводе,1195
вдовец / вдова,959
гражданский брак,4163
женат / замужем,12343


57.48952026082906

Из представленных данных следует, что  максимальное количество заемщиков с семейным положением "женат / замужем", которое составляет 57,5% от всех заемщиков. 
Cгруппированные данные по заемщикам, которые вышли на просрочку не изменятся и также составят 8.1%, как и в первой гипотезе.

Теперь сформируем сводную таблицу, которая посчитает зависимость между семейным положением и возвратом кредита в срок.

In [54]:
make_dependence('family_status')

debt,percent
family_status,
Не женат / не замужем,9.750890
в разводе,7.112971
вдовец / вдова,6.569343
гражданский брак,9.320202
женат / замужем,7.542737


Как видно из данных, приведенных в таблице выше, максимальный процент просроченной задолженности у 
заемщиков с семейным положением "Не женат / не замужем" (9.8%), минимальный процент просроченной задолженности 
у заемщиков с семейным положением "вдовец / вдова"/ (6.5%).

Рассмотрим как влияет семейное положение на возврат кредита в срок у должников с разным уровнем образования.

In [55]:
make_double_dependence('family_status', 'education')

debt                    percent                                            \
education                высшее  начальное неоконченное высшее    среднее   
family_status                                                               
Не женат / не замужем  5.714286   8.823529            9.326425  11.648846   
в разводе              4.746835  10.000000           12.903226   7.738815   
вдовец / вдова         5.147059   0.000000           15.384615   6.931964   
гражданский брак       5.765199  19.642857            8.433735  10.311349   
женат / замужем        5.098684  10.638298            8.797654   8.290802   

debt                                  
education             ученая степень  
family_status                         
Не женат / не замужем            0.0  
в разводе                        0.0  
вдовец / вдова                   NaN  
гражданский брак                 NaN  
женат / замужем                  0.0

Как видно из таблицы выше, просрочка отсутствет у должников с ученой степенью. 
При этом, у должников с высшим образованием процент просроченной задолженности также ниже (от 0% до 5.7%) процента просроченнной задолженности всех заемщиков (8.1%). Кроме того, у должников с неоконченным высшим образованием процент просроченной задолженности возрастает с увеличением количества детей.

Рассмотрим как влияет наличие детей на возврат кредита в срок у должников разного пола.

In [56]:
make_double_dependence('family_status', 'gender')

debt                    percent           
gender                        F          M
family_status                             
Не женат / не замужем  6.824754  14.431082
в разводе              6.517094   9.266409
вдовец / вдова         5.752212  20.000000
гражданский брак       8.158263  11.859220
женат / замужем        6.827258   8.755459

Как видно из данных, приведенных в таблице выше, у заемщиков женского пола, вне зависимости от семейного статуса процент просроченной задолженности ниже, чем у заемщиков мужского пола.
У заемщиков мужского пола, наибольший процент просроченной задолженности приходится на семейный статус "вдовец / вдова", а у заемщиков жеского пола на семейный статус "гражданский брак". Наименьший процент просроченной задолженности приходится на заемщиков мужского пола с семейным статусом "женат / замужем", а у заемщиков женского пола на семейный статус "вдовец / вдова".

Рассмотрим влияние семейного статуса на возврат кредита в срок у должников разного возраста.

In [57]:
make_double_dependence('family_status', 'dob_years_group')

debt                     percent                                
dob_years_group            до 22      до 30      до 55 остальные
family_status                                                   
Не женат / не замужем  12.359551  11.099138  10.310022  4.187192
в разводе               0.000000  14.782609   6.364749  6.130268
вдовец / вдова               NaN  14.285714   6.590258  6.467662
гражданский брак        4.545455  11.155914   9.808343  5.882353
женат / замужем        14.285714  10.329171   7.550870  5.333893

Как видно из данных, приведенных в таблице выше, наибольший процент просроченной задолженности приходится на заемщиков с семейным статусом "вдовец / вдова" в возрасте до 21 года, при этом, наименьший просроченной задолженности приходится на заемщиков с семейным статусом "Не женат / не замужем" в возрасте от 55 лет.

**Вывод**

В ходе анализа наличия зависимости между наличием детей и возвратом кредита в срок, установлено следующее.

Максимальное количество заемщиков в представленном датасете  - это заемщики с семейным положением "женат / замужем", их количество составляет 57,5%. 

При этом, максимальный процент просроченной задолженности у заемщиков с семейным положением "Не женат / не замужем" (9.8%), минимальный процент просроченной задолженности у заемщиков с семейным положением "вдовец / вдова"/ (6.5%).

Cгруппированные данные по заемщикам, которые вышли на просрочку не изменятся и также составят 8.1%, как и в первой гипотезе.


Кроме того, в ходе проведенного анализа установлено, что:
1. просрочка отсутствет у должников с ученой степенью. При этом, у должников с высшим образованием процент просроченной задолженности также ниже (от 0% до 5.7%) процента просроченнной задолженности всех заемщиков (8.1%). Кроме того, у должников с неоконченным высшим образованием процент просроченной задолженности возрастает с увеличением количества детей.


2. наибольший процент просроченной задолженности приходится на заемщиков с семейным статусом "вдовец / вдова" в возрасте до 21 года, при этом, наименьший процент просроченной задолженности приходится на заемщиков с семейным статусом "Не женат / не замужем" в возрасте от 55 лет.

3. у заемщиков женского пола, вне зависимости от семейного статуса, процент просроченной задолженности ниже, чем у заемщиков мужского пола.

4. У заемщиков мужского пола, наибольший процент просроченной задолженности приходится на семейный статус "вдовец / вдова", а у заемщиков жеского пола на семейный статус "гражданский брак". 

5. Наименьший процент просроченной задолженности приходится на заемщиков мужского пола с семейным статусом "женат / замужем", а у заемщиков женского пола на семейный статус "вдовец / вдова".

6. просрочка отсутствет у должников с ученой степенью. При этом, у должников с высшим образованием процент просроченной задолженности также ниже (от 0% до 5.7%) процента просроченнной задолженности всех заемщиков (8.1%). 

7. у должников с неоконченным высшим образованием процент просроченной задолженности возрастает с увеличением количества детей.

По результатам проведенного анализа установлено, что гипотеза о зависимости между семейным положением и возвратом кредита в срок не подтверждается. Если зависимость и существует, то на основной массе должников указанного датасета она не заметна.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

При рассмотрении третьей гипотезы проверим, есть ли зависимость между уровнем дохода и возвратом кредита в срок. 
Для этого сначала оценим количество заемщиков для заемщиков с разным уровнем дохода.

In [58]:
min_percent = df.groupby('total_income_group')[['debt']].count()
display(min_percent)
display(min_percent['debt'][1] / max_percent['debt'].sum() * 100)

,debt
total_income_group,
до 150 тыс.,11632
до 22 тыс.,5
свыше 1 млн,9833


0.02328830926874709

Из представленных данных следует, что  заемщики с минимимальным уровнем дохода (до 50 тыс.) составляют 0,02% от всех заемщиков. 
Cгруппированные данные по заемщикам, которые вышли на просрочку не изменятся и также составят 8.1%, как и в первой гипотезе.

Теперь сформируем сводную таблицу, которая посчитает зависимость между уровнем дохода и возвратом кредита в срок.

In [59]:
make_dependence('total_income_group')

debt,percent
total_income_group,
до 150 тыс.,8.399243
до 22 тыс.,20.000000
свыше 1 млн,7.759585


Как видно из данных, приведенных в таблице выше, самый большой процент просроченной задолженности у заемщиков с самым маленьким доходом (до 50 тыс.). При этом с увеличением суммы дохода, по сформированным в процессе исследования группам, у заемщиков уменьшается процент просроченной задолженности.

In [60]:
make_dependence('total_income_kvant')

debt,percent
total_income_kvant,
до 108 тыс.,7.954545
до 143 тыс.,8.799417
до 196 тыс.,8.541468
свыше 196,7.134873


Рассмотрим как влияет уровень дохода на возврат кредита в срок у должников с разным уровнем образования.

In [61]:
make_double_dependence('total_income_group', 'education')

debt                 percent                                            \
education             высшее  начальное неоконченное высшее    среднее   
total_income_group                                                       
до 150 тыс.         5.476773  10.714286            8.215297   9.020476   
до 22 тыс.               NaN        NaN                 NaN  20.000000   
свыше 1 млн         5.177792  11.627907            9.974425   8.914918   

debt                               
education          ученая степень  
total_income_group                 
до 150 тыс.                   0.0  
до 22 тыс.                    NaN  
свыше 1 млн                   0.0

Как видно из таблицы выше, максимальный процент просроченной задолженности у заемщиков
с уровнем дохода до 50 тыс. и составляет 20%.

Рассмотрим как влияет уровень дохода на возврат кредита в срок у должников разного пола.

In [62]:
make_double_dependence('total_income_group', 'gender')

debt                  percent           
gender                      F          M
total_income_group                      
до 150 тыс.          7.332787  11.340872
до 22 тыс.          50.000000   0.000000
свыше 1 млн          6.496725   9.464627

Как видно из данных, приведенных в таблице выше, у заемщиков, доход которых больше,
вне зависимости от пола, процент просроченной задолженности меньше.

**Вывод**

Из представленных данных следует, что заемщики с минимимальным уровнем дохода (до 50 тыс.) составляют 0,02% от всех заемщиков. 

Самый большой процент просроченной задолженности (20%) у заемщиков с самым маленьким доходом (до 50 тыс.). При этом чем больше сумма дохода у заемщика, по сформированным в процессе исследования группам, тем меньше процент просроченной задолженности.

По результатам проведенного анализа установлена зависимость между уровнем дохода заемщика, по сформированным в процессе исследования группам, и возвратом кредита в срок.

Кроме того, в ходе исследования данных установлено, что у заемщиков, доход которых больше, вне зависимости от пола, процент просроченной задолженности меньше.

Таким образом, более детальный анализ данных также подтверждает зависимость между уровнем дохода и возвратом кредита в срок.

Гипотеза о зависимости между уровнем дохода и возвратом кредита в срок подтверждается.

- Как разные цели кредита влияют на его возврат в срок?

При рассмотрении четвертой гипотезы проверим, как разные цели кредита влияют на его возврат в срок. 
Для этого сначала проведем расчет, сколько всего заемщиков с разными целями кредита.

Оценим количество заемщиков для заемщиков с разными целями кредита.

In [63]:
min_percent = df.groupby('purpose')[['debt']].count()
display(min_percent)
display(min_percent['debt'][1] / max_percent['debt'].sum() * 100)

,debt
purpose,
на проведение свадьбы,2335
операции с недвижимостью,10813
покупка автомобиля,4308
получение образования,4014


50.36329762459245

Из представленных данных следует, что  целью 50,4% заемщиков являлись операци с недвижимостью. 
Cгруппированные данные по заемщикам, которые вышли на просрочку не изменятся и также составят 8.1%, как и в первой гипотезе.

Теперь сформируем сводную таблицу, которая посчитает зависимость между целью кредита и возвратом кредита в срок.

In [64]:
make_dependence('purpose')

debt,percent
purpose,
на проведение свадьбы,7.965739
операции с недвижимостью,7.232036
покупка автомобиля,9.354689
получение образования,9.217738


Как видно из данных, приведенных в таблице выше, наибольший процент просроченной задолженности приходится на покупку  автомобиля и составляет 9.4%. 
Наименьший процент просроченной задолженности приходится на операции с недвижимостью.




Рассмотрим как влияет цель кредита на возврат кредита в срок у должников с разным уровнем образования.

In [65]:
make_double_dependence('purpose', 'education')

debt                       percent                                            \
education                   высшее  начальное неоконченное высшее    среднее   
purpose                                                                        
на проведение свадьбы     4.492512  14.285714            7.500000   9.163592   
операции с недвижимостью  4.813433  11.111111            8.947368   7.930037   
покупка автомобиля        6.467181  10.909091           10.457516  10.248042   
получение образования     5.888651   9.090909            9.160305  10.300726   

debt                                     
education                ученая степень  
purpose                                  
на проведение свадьбы               NaN  
операции с недвижимостью            0.0  
покупка автомобиля                  NaN  
получение образования               0.0

Как видно из данных, приведенных в таблице выше, у заемщиков с разным уровнем образования, 
наибольший процент просроченной задолженности по кредитам, целью которых является покупка автомобиля.

Рассмотрим как влияет цель кредита на возврат кредита в срок у должников с разным типом дохода.

In [66]:
make_double_dependence('purpose', 'income_type')

debt                         percent                                  \
income_type              безработный в декрете госслужащий компаньон   
purpose                                                                
на проведение свадьбы            NaN       NaN    4.402516  9.867173   
операции с недвижимостью        50.0       NaN    4.774536  6.556733   
покупка автомобиля               NaN     100.0    7.692308  8.079848   
получение образования            NaN       NaN    8.139535  7.547170   

debt                                                                   
income_type              пенсионер предприниматель  сотрудник студент  
purpose                                                                
на проведение свадьбы     5.841121             0.0   8.360656     NaN  
операции с недвижимостью  4.865151             0.0   8.652306     0.0  
покупка автомобиля        6.407035             NaN  11.228716     NaN  
получение образования     6.648199             NaN  11.009615     NaN

Как видно из данных, приведенных в таблице выше, наибольший процент просроченной задолженности 
приходится на покупку автомобиля  безработными и составляет 100%.

Рассмотрим как влияет цель кредита на возврат кредита в срок у должников разного пола.

In [67]:
make_double_dependence('purpose', 'gender')

debt                       percent           
gender                           F          M
purpose                                      
на проведение свадьбы     6.870229  10.222805
операции с недвижимостью  6.265876   9.068956
покупка автомобиля        8.211041  11.618257
получение образования     7.758621  12.109955

Как видно из данных, приведенных в таблице выше, наименьший процент просроченной задолженности 
приходится на операции с недвижимостью как у мужчин, так и у женщин.

**Вывод**

Из представленных данных следует, что целью 50,4% заемщиков являлись операци с недвижимостью. 
Наибольший процент просроченной задолженности приходится на покупку автомобиля и составляет 9.4%. Наименьший процент просроченной задолженности приходится на операции с недвижимостью.

По результатам более детального анализа также установлено:

1. наибольший процент просроченной задолженности приходится на покупку автомобиля  безработными и составляет 100%.

2. наименьший процент просроченной задолженности приходится на операции с недвижимостью как у мужчин, так и у женщин.

3. у заемщиков с разным уровнем образования, наибольший процент просроченной задолженности приходится на кредиты, целью которых является покупка автомобиля.


По результатам проведенного анализа установлена зависимость между целью кредита и возвратом кредита в срок. 
Так, наибольшлий процент просроченной задолженности приходится на кредиты, целью которых является покупка автомобиля, а наименьший процент просроченной задолженности приходится на кредиты, целью которых являются операции с недвижимостью.


## Шаг 4. Общий вывод


Исходя из вышеизложенного, в ходе проведения исследования надежности заемщиков установлено, что уровень дохода заемщика, а также цель кредита влияют на вероятность своевременного исполнения обязательств по кредиту.
Чем больше у заемщика доход, тем меньше процент просроченной задолженности. 
Кроме того, заемщики, целью кредита которых являются операции с недвижимостью, чаще исполняют обязательства в срок, чем заемщики, целью кредита которых является покупка автомобиля. 